# Pulling Data

Use three months of data from Seattle, Oakland, and Louisville
- start date of "2022-09-01"
- end date of "2022-12-01"

In [1]:
from cdp_data import CDPInstances, datasets
import pandas as pd

In [2]:
# Get a dataset of "city council sessions" for Seattle 2022-09-01 to 2022-12-01
seattle_transcripts_sep_2022_to_dec_2022 = datasets.get_session_dataset(
    CDPInstances.Seattle,  # specify the city (or county) council we want data from
    start_datetime="2022-09-01",  # YYYY-MM-DD format
    end_datetime="2022-12-01",  # YYYY-MM-DD format
    store_transcript=True,  # store transcripts locally for fast file reading
    store_transcript_as_csv=True,  # store transcripts as CSVs for easy pandas reading
)

# Get a dataset of "city council sessions" for Oakland 2022-09-01 to 2022-12-01
oakland_transcripts_sep_2022_to_dec_2022 = datasets.get_session_dataset(
    CDPInstances.Oakland,  # specify the city (or county) council we want data from
    start_datetime="2022-09-01",  # YYYY-MM-DD format
    end_datetime="2022-12-01",  # YYYY-MM-DD format
    store_transcript=True,  # store transcripts locally for fast file reading
    store_transcript_as_csv=True,  # store transcripts as CSVs for easy pandas reading
)

# Get a dataset of "city council sessions" for Louisville 2022-09-01 to 2022-12-01
louisville_transcripts_sep_2022_to_dec_2022 = datasets.get_session_dataset(
    CDPInstances.Louisville,  # specify the city (or county) council we want data from
    start_datetime="2022-09-01",  # YYYY-MM-DD format
    end_datetime="2022-12-01",  # YYYY-MM-DD format
    store_transcript=True,  # store transcripts locally for fast file reading
    store_transcript_as_csv=True,  # store transcripts as CSVs for easy pandas reading
)

Fetching each model attached to event_ref:   0%|          | 0/17 [00:00<?, ?it/s]

Fetching transcripts:   0%|          | 0/17 [00:00<?, ?it/s]

Converting and storing each transcript as a CSV: 17it [00:08,  1.98it/s]


Fetching each model attached to event_ref:   0%|          | 0/18 [00:00<?, ?it/s]

Fetching transcripts:   0%|          | 0/18 [00:00<?, ?it/s]

Converting and storing each transcript as a CSV: 18it [00:09,  1.94it/s]


Fetching each model attached to event_ref:   0%|          | 0/77 [00:00<?, ?it/s]

Fetching transcripts:   0%|          | 0/77 [00:00<?, ?it/s]

Converting and storing each transcript as a CSV: 77it [00:32,  2.35it/s]


# Recognizing NERs using SpaCy

In [3]:
import spacy 

nlp_spacy = spacy.load("en_core_web_sm")

# The NERs we are interested in
ners = ['PERSON', 'ORG', 'NORP']

In [7]:
# seattle
seattle_rows = []
seattle_sentences_dfs = {}

for i, session in seattle_transcripts_sep_2022_to_dec_2022.iterrows():
    
    seattle_sentences_df = pd.read_csv(session.transcript_as_csv_path)
    # drop rows if text column is empty
    seattle_sentences_df = seattle_sentences_df.dropna(subset=["text"])
    
    # to keep session_id
    seattle_sentences_dfs[session.transcript_as_csv_path] = seattle_sentences_df.assign(session_id = session.id)
    
    for t in seattle_sentences_df.index:
        doc = nlp_spacy(str(seattle_sentences_df.text[t]))
        for ent in doc.ents:
            if ent.label_ in ners:
                seattle_rows.append([session.id, t, ent.label_, 
                                     ent.text, 'Seattle'])
    
            
seattle_df = pd.DataFrame(seattle_rows, columns=["session_id", "sentence_index", "named_entity_type", 
                                                 "entity", "city"])

seattle_all_sentence_df = pd.concat(seattle_sentences_dfs).reset_index()


# oakland
oakland_rows = []
oakland_sentences_dfs = {}

for i, session in oakland_transcripts_sep_2022_to_dec_2022.iterrows():
    
    oakland_sentences_df = pd.read_csv(session.transcript_as_csv_path)
    # drop rows if text column is empty
    oakland_sentences_df = oakland_sentences_df.dropna(subset=["text"])

    # to keep session_id
    oakland_sentences_dfs[session.transcript_as_csv_path] = oakland_sentences_df.assign(session_id = session.id)
        
    for t in oakland_sentences_df.index:
        doc = nlp_spacy(str(oakland_sentences_df.text[t]))
        for ent in doc.ents:
            if ent.label_ in ners:
                oakland_rows.append([session.id, t, ent.label_, 
                                     ent.text, 'Oakland'])


oakland_df = pd.DataFrame(oakland_rows, columns=["session_id", "sentence_index", "named_entity_type", 
                                                 "entity", "city"])


oakland_all_sentence_df = pd.concat(oakland_sentences_dfs).reset_index()


# louisville
louisville_rows = []
louisville_sentences_dfs = {}

for i, session in louisville_transcripts_sep_2022_to_dec_2022.iterrows():
    
    louisville_sentences_df = pd.read_csv(session.transcript_as_csv_path)
    # drop rows if text column is empty
    louisville_sentences_df = louisville_sentences_df.dropna(subset=["text"])
    
    # to keep session_id
    louisville_sentences_dfs[session.transcript_as_csv_path] = louisville_sentences_df.assign(session_id = session.id)

    for t in louisville_sentences_df.index:
        doc = nlp_spacy(str(louisville_sentences_df.text[t]))
        for ent in doc.ents:
            if ent.label_ in ners:
                louisville_rows.append([session.id, t, ent.label_, 
                                        ent.text, 'Louisville'])


louisville_df = pd.DataFrame(louisville_rows, columns=["session_id", "sentence_index", "named_entity_type", 
                                                       "entity", "city"])

louisville_all_sentence_df = pd.concat(louisville_sentences_dfs).reset_index()

# Combining all NER results 

- Combined `seattle_df`, `oakland_df`, and `louisville_df` into 1 dataframe `all_cities_df`.

- Combined `seattle_all_sentence_df`, `oakland_all_sentence_df`, and `louisville_all_sentence_df` into 1 dataframe `all_cities_sentences_df`.


In [8]:
all_cities_df = pd.concat([seattle_df, oakland_df, louisville_df])

all_cities_sentences_df = pd.concat([seattle_all_sentence_df, 
                                     oakland_all_sentence_df, 
                                     louisville_all_sentence_df])

# Normalization

- Divide index by total number of sentences == % of meeting where sentence appears

- Add a new column to all_cities_df 'percentages' indicating when an entity is recognized in the meeting.